In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LinearRegression

from sqlalchemy import create_engine

In [ ]:
data = pd.read_excel('data.xlsx', parse_dates=['period'])

In [ ]:
sber_values = {}
for k, g in data.groupby(by='period'):
    sber_values.update({k: np.sum(g['Parametr_value'])})

sber = pd.Series(sber_values)


In [ ]:
keys = ['ТБ', 'dept_id', 'Group_500', 'Валюта']
backtest_size = 6 # backtest_size
x_test = sber[-6:]
x_train = sber[~sber.index.isin(x_test.index)][-6:]
results = []

for key, group in data.groupby(by = keys):
    group = group[['period', 'Parametr_value']].\
            set_index('period').sort_index().squeeze().asfreq('M')
        
    y_test = group[x_test.index]
    y_train = group[x_train.index]
        
    linear_reg = LinearRegression()
    linear_reg.fit(x_train.values, y_train.values)
    y_pred = linear_reg.predict(x_test.values)
    break

In [ ]:
linear_reg.fit(x_train.to_sparse(), y_train.to_sparse())

In [ ]:
MAPE = abs(y_pred / y_test - 1).mean()
sMAPE = (f'{MAPE:0.2%}')

plt.suptitle('MAPE: ' + sMAPE)
group.plot()
pd.Series(y_pred.reshape(1, -1)[0],
          index=group.index[-len(y_test):]).plot()

plt.show()

In [ ]:
results_df.groupby(by=keys).error.mean().mean()

In [ ]:
engine = create_engine('mssql+pyodbc://MSSQL_DSN')
engine.execute('USE Model_Passiv_FL_ML')        

results_df.to_sql(name = 't_for_fedor_my_gosb_fast_result', 
                  con =  engine, schema = 'dbo',
                  index = False,
                  if_exists='append')